In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score, recall_score

In [2]:
## 设置字符集，防止中文乱码
mpl.rcParams['font.sans-serif']=[u'simHei']
mpl.rcParams['axes.unicode_minus']=False

In [3]:
# jupyter展示图片，非内嵌显示
# tk: 显示出来，inline：内嵌显示，默认为inline
%matplotlib tk

In [4]:
# 1. 文件数据读取
df = pd.read_csv('../data/result_process02', sep=',')
# 如果有某值为nan，进行删除操作
df.dropna(axis=0, how='any', inplace=True)
df.head(10)

,label,has_date,jieba_cut_content,content_sema
0,1.0,1,非 财务 纠淼 牟 莆 窆 芾 - （ 沙盘 模拟 ） - - ...,8.456151
1,0.0,1,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ...,7.486084
2,1.0,1,尊敬 的 贵 公司 ( 财务 / 经理 ) 负责人 您好 ！ 我 ...,7.175171
3,1.0,1,贵 公司 负责人 ( 经理 / 财务 ） 您好 ： 深圳市 华龙 公...,7.565682
4,1.0,1,这是 一封 HTML 格式 信件 ！ - - - - - - - ...,2.063409
5,1.0,1,TO ： 贵 公司 经理 、 财务 您好 ！ 深圳市 春洋 贸易 有...,7.143747
6,0.0,1,那 他 为什么 不 愿意 起诉 ， 既然 这样 了 ！ 起诉 后 ...,4.807568
7,1.0,1,尊敬 的 负责人 （ 经理 ／ 财务 ） ： 您好 ！ 我 是 深...,6.593684
8,1.0,1,您好 以下 是 特别 为 阁下 发 的 香港 信息 ( 图片 ...,7.611074
9,0.0,1,我 觉得 ， 负债 不要紧 ， 最 重要 的 是 能 负得起 这个 ...,7.041340


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64284 entries, 0 to 64619
Data columns (total 4 columns):
label                64284 non-null float64
has_date             64284 non-null int64
jieba_cut_content    64284 non-null object
content_sema         64284 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.5+ MB


In [6]:
# 2. 数据分割
x_train, x_test, y_train, y_test = train_test_split(df[['has_date','jieba_cut_content','content_sema']], df['label'], test_size=0.2, random_state=0)
print("训练数据集大小:%d" % x_train.shape[0])
print("测试数据集大小:%d" % x_test.shape[0])

训练数据集大小:51427
测试数据集大小:12857


In [7]:
x_train.head(5)

,has_date,jieba_cut_content,content_sema
18810,1,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.849239
61632,1,声音 变换器 ： 适用 于 不同 型号 的 手机 ， 电话 ， 网络...,7.242064
5822,1,http : / / tengyingge . blogchina . co...,8.289843
5881,1,刚 开始 ， 是 会 有点 不 习惯 的 。 慢慢来 。 ps ：...,7.063309
26338,1,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.700159


In [8]:
# 3. 开始模型训练
# 3.1 特征工程，将文本数据转换为数值型数据
vectorizer = CountVectorizer()
transfromer = TfidfTransformer(norm='l2', use_idf=True)
svd = TruncatedSVD(n_components=20)
jieba_cut_content = list(x_train['jieba_cut_content'].astype('str'))
vectorizer_model = vectorizer.fit(jieba_cut_content)
df0 = vectorizer_model.transform(jieba_cut_content)
transfromer_model = transfromer.fit(df0)
df1 = transfromer_model.transform(df0)
svd_model = svd.fit(df1)
df2 = svd_model.transform(df1)
data = pd.DataFrame(df2)
print(data.head(5))
print(data.info())

         0         1         2         3         4         5         6   \
0  0.340289  0.220017 -0.237459 -0.039885 -0.070265  0.002604 -0.160720   
1  0.014968  0.064740  0.027002 -0.004669  0.008155  0.007462  0.016359   
2  0.011862  0.072032  0.047005  0.001421 -0.002079  0.035894  0.009225   
3  0.019714  0.117440  0.132451  0.060414 -0.103506 -0.074869 -0.012184   
4  0.346257  0.215341 -0.234010 -0.043062 -0.071182  0.000061 -0.162542   

         7         8         9         10        11        12        13  \
0 -0.007678  0.200393 -0.008583 -0.197268  0.039608 -0.063572  0.440220   
1  0.046098  0.029387 -0.000211  0.007908  0.023442  0.041765 -0.012593   
2 -0.012259 -0.016788  0.000296  0.001241 -0.004071 -0.002510 -0.002772   
3 -0.005446 -0.027818  0.000383 -0.016861 -0.013143 -0.002746  0.017886   
4 -0.006943  0.197694 -0.008343 -0.192771  0.037944 -0.059985  0.428308   

         14        15        16        17        18        19  
0 -0.458792  0.208671  0.094317 -0

In [9]:
# 3.2 数据合并
data['has_date'] = list(x_train['has_date'])
data['content_sema'] = list(x_train['content_sema'])
print(data.head(5))
print(data.info())

          0         1         2         3         4         5         6  \
0  0.340289  0.220017 -0.237459 -0.039885 -0.070265  0.002604 -0.160720   
1  0.014968  0.064740  0.027002 -0.004669  0.008155  0.007462  0.016359   
2  0.011862  0.072032  0.047005  0.001421 -0.002079  0.035894  0.009225   
3  0.019714  0.117440  0.132451  0.060414 -0.103506 -0.074869 -0.012184   
4  0.346257  0.215341 -0.234010 -0.043062 -0.071182  0.000061 -0.162542   

          7         8         9      ...             12        13        14  \
0 -0.007678  0.200393 -0.008583      ...      -0.063572  0.440220 -0.458792   
1  0.046098  0.029387 -0.000211      ...       0.041765 -0.012593  0.013109   
2 -0.012259 -0.016788  0.000296      ...      -0.002510 -0.002772 -0.003071   
3 -0.005446 -0.027818  0.000383      ...      -0.002746  0.017886  0.002681   
4 -0.006943  0.197694 -0.008343      ...      -0.059985  0.428308 -0.448800   

         15        16        17        18        19  has_date  content_sem

In [10]:
%%time
forest = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=3, random_state=0)
model = forest.fit(data, y_train)

Wall time: 6.26 s


In [11]:
# 4.1 对测试数据进行特征转换
jieba_cut_content_test = list(x_test['jieba_cut_content'].astype('str'))
data_test = pd.DataFrame(svd_model.transform(transfromer_model.transform(vectorizer_model.transform(jieba_cut_content_test))))
data_test['has_date'] = list(x_test['has_date'])
data_test['content_sema'] = list(x_test['content_sema'])
print(data_test.head(5))
print(data_test.info())

          0         1         2         3         4         5         6  \
0  0.537283  0.009051 -0.087582  0.028987  0.000918 -0.032667 -0.157513   
1  0.210902  0.131679 -0.090070 -0.086948 -0.033331  0.039695  0.007113   
2  0.045151  0.277281  0.233902  0.051069 -0.074015 -0.016345  0.010681   
3  0.032907  0.163067  0.113490  0.002468  0.053723  0.008259  0.004287   
4  0.035278  0.065206  0.026084 -0.023764  0.072921 -0.005833 -0.003143   

          7         8         9      ...             12        13        14  \
0  0.009324 -0.134734  0.009492      ...       0.081574  0.007025 -0.014326   
1 -0.007900  0.011396 -0.000252      ...      -0.025573  0.083579 -0.054450   
2 -0.013751  0.100075 -0.002574      ...      -0.066010 -0.038171 -0.029468   
3 -0.019831  0.117836 -0.002735      ...      -0.061469 -0.058450 -0.013349   
4 -0.007768  0.007772  0.000809      ...      -0.001760 -0.004434 -0.002506   

         15        16        17        18        19  has_date  content_sem

In [12]:
# 4.2 对测试数据进行预测
y_predict = model.predict(data_test)

In [13]:
# 5. 效果评估
print("准确率为:%.5f" % precision_score(y_test, y_predict))
print("召回率为:%.5f" % recall_score(y_test, y_predict))
print("F1值为:%.5f" % f1_score(y_test, y_predict))

准确率为:0.94326
召回率为:0.98925
F1值为:0.96571
